# Letterments kata

The target of this kata is to convert a word to a list of possible strings with [periodic table element names](http://www.ptable.com/) from the original word's letters, for example:
```
letterments('cab') = ['Calcium Boron']
```
The result of the function is a list, since there are words that might have several solutions.
In case there is no possible result at all for a word, the function will return an empty list.

Solution to this kata should provide tests, further than the provided cases here.
In order to do that, there is a [pretty extensive list of words that might be translated to elemens](https://gist.github.com/jeffThompson/7789182), which would be used.

Let's see some examples:

In [ ]:
cases = [
    ('c', ['Carbon']),
    ('z', []),
    ('al', ['Aluminum']),
    ('bo', ['Boron Oxygen']),
    ('ab', []),
    ('cab', ['Calcium Boron']),
    ('joy', []),
    ('bin', ['Boron Indium', 'Bismuth Nitrogen', 'Boron Iodine Nitrogen']),
]

def letterments_example(word):
    for case in cases:
        if case[0] == word:
            return case

In [ ]:
for case in cases:
    print("letterments('{}') = {}".format(case[0], letterments_example(case[0])))

## Tooling for this dojo

In first place, this goodie to run our `unittest` test class has been included:

In [ ]:
import sys
import unittest

def test(testClass):
    suite = unittest.TestLoader().loadTestsFromTestCase(testClass)
    unittest.TextTestRunner(verbosity=1, stream=sys.stderr).run(suite)

To ease to work with periodic table elements, usage of the `periodictable` module is recommended:

In [ ]:
import periodictable

This way, a complete set of symbols and elements are already available...

... which can be queried by the `get_element_name` function.
This function gets a symbol, and returns the capitalized element name if symbol already exists, or `None`, otherwise.

**WARNING**: For some weird reason, `periodictable` includes Deuterium (D) as an element:

In [ ]:
def get_element_name(symbol):
    try:
        return eval("periodictable.{}.name".format(symbol.capitalize())).capitalize()
    except AttributeError:
        pass

In [ ]:
for case in cases:
    print("get_element_name('{}') = {}".format(case[0], get_element_name(case[0])))
print("get_element_name('d') = {}".format(get_element_name('d')))

## Approaches

Some approaches to solve this kata could be already discarded, due to complexity of the problem, but some possible options were suggested:

* Brute force by iteration: Iterating over the letters in the word, to replace.
* Brute force by split: Creating a list of possible divisions of the word, like `[['w', 'o', 'r', 'd'], []`, and iterating over these, replacing each part by elements, and discarding the incomplete ones.
* Brute force by replacing: Using `str.replace` or regular expressions, to replace symbols by element names incrementally.
* Using concurrency to check every possibility.
* [Backtracking](https://en.wikipedia.org/wiki/Backtracking)

In [ ]:
word = 'accepts'
a, b, *c = word
print("{} {} {}".format(a, b, c))


In [ ]:
def solutions(f):
    memo = []
    def wrapper(word, result):
        if result not in memo:
            memo.append(result)
            f(word, result)

def letterments(word, result=None):
    if len(word) > 2:
        a, b, *c = word
    elif len(word) > 1:
        a, b = word
        c = []
    else:
        a = word
        b = ''
        c = []
    if result is None:
        result = []
    solution = 0
    second_branch = solution
    if not a:
        return result
    element = get_element_name(a)
    print("current{}: {} {}".format(len(result), a, element))
    result.append([])
    if element is not None:
        result[solution].append(element)
        if len(word) > 0:
            result[solution].extend(letterments("".join([b] + c), result))
    if not b:
        return result
    element = get_element_name(a + b)
    print("current{}: {} {}".format(len(result), a + b, element))
    if element is not None:
        second_branch = solution + 1
        result.append([])
        result[second_branch].append(element)
        if len(word) > 0:
            result[second_branch].extend(letterments("".join(c), result))
    return result

for solution in letterments('bicarbs'):
    print(solution)

#for solution in letterments('accrual'):
#    print(solution)

In [7]:
import copy 
import periodictable

def get_element_name(symbol):
    try:
        return eval("periodictable.{}.name".format(symbol.capitalize())).capitalize()
    except AttributeError:
        pass

def memoize(obj):
    cache = {}
    def memoizer(*args, **kwargs):
        word, result, solution = args
        index = "{}-{}".format(word, solution)
        if index not in cache:
            cache[index] = obj(word, result, solution)
        return cache[index]
    return memoizer

@memoize
def letterments(word, result, solution):
    if len(word) > 2:
        a, b, *c = word
    elif len(word) > 1:
        a, b = word
        c = []
    else:
        a = word
        b = ''
        c = []
    if result is None:
        result = [[]]
    if not a:
        return result
    element = get_element_name(a)
    element_2 = get_element_name(a + b)
    
    if element is not None:
        
        if element_2 is not None:
            result.append(copy.deepcopy(result[solution]))
            result[solution+1].append(element_2)
            if len(word) > 0:
                result[solution+1].extend(letterments("".join(c), result, solution + 1))
        result[solution].append(element)
        if len(word) > 0:
            result[solution].extend(letterments("".join([b] + c), result, solution))
    else:    
        if element_2 is not None:
            result[solution].append(element_2)
            if len(word) > 0:
                result[solution].extend(letterments("".join(c), result, solution))
        else:
            del result[-1]
    #print(result[solution])
    #print("current{}: {} {}".format(len(result), a, element))
    return result

res = letterments('bicarbs', None, 0)

print(res[0])

#for solution in letterments('bicarbs'):
#    print(solution)

#for solution in letterments('accrual'):
#    print(solution)

['Boron', 'Iodine', 'Carbon', 'Argon', 'Boron', 'Sulfur', [...], ['Bismuth', 'Carbon', 'Argon', 'Boron', 'Sulfur', [...], [...], ['Bismuth', 'Calcium', 'Rubidium', 'Sulfur', [...], [...], [...], ['Bismuth', 'Calcium', 'Rubidium', 'Sulfur', [...], [...], [...], [...]], [...], [...], [...], ['Bismuth', 'Calcium', 'Rubidium', 'Sulfur', [...], [...], [...], [...]], [...], [...], [...], ['Bismuth', 'Calcium', 'Rubidium', 'Sulfur', [...], [...], [...], [...]], 'Sulfur', [...], [...], [...], ['Bismuth', 'Calcium', 'Rubidium', 'Sulfur', [...], [...], [...], [...]], ['Bismuth', 'Carbon', 'Argon', 'Boron']], ['Bismuth', 'Calcium', 'Rubidium', 'Sulfur', [...], [...], ['Bismuth', 'Calcium', 'Rubidium', 'Sulfur', [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], 'Sulfur', [...], [...], [...], [...], ['Bismuth', 'Carbon', 'Argon', 'Boron']], [...]], ['Bismuth', 'Carbon', 'Argon', 'Boron'], [...], [...], ['Bismuth', 'Calcium', 'Rubidium', 'Sulfur', [...], [...], [...